In [1]:
using LinearAlgebra, CSV, JuMP, DataFrames, Cbc, Dates

### Model 1: Maximum coverage with a limit of areas that can be covered, x and y not within .01 of existing electricity

$max \sum{b_i*v_i}$

$\sqrt{(ex_j-x_i)^2+(ey_j-y_i)^2} \geq min\_dist$

$\sum{b_i} \leq max\_units$

$i \in \{1...len(clusters)\}$

$j \in \{1...len(elecricity)\}$

In [2]:
function model1opt(max_units = 25, min_dist = 1, filter_density = 0.006)
    model1 = Model(Cbc.Optimizer)

    df= CSV.read("Data_With_Dist.csv", copycols=true)
#     electricity = CSV.read("electricity.csv", copycols=true)
    fdf = filter(row -> row[:value] > filter_density, df)

    m = size(fdf)[1]
#     n = size(electricity)[1]

    @variable(model1, b[1:m], Bin)

    @constraint(model1,sum(b[i] for i=1:m)<=max_units)

    for i=1:m
        @constraint(model1, fdf[9][i] >= min_dist*b[i])
    end

    @objective(model1, Max, sum(fdf[8][i]*b[i] for i = 1:m))

    model1;
    
    optimize!(model1)
    obj_val = objective_value(model1)
    
    soln = value.(b)
    sol = [fdf.Location[i] for (i,n) in enumerate(soln) if n == 1]
    
    return (max_units, min_dist, filter_density, obj_val, length(sol), sol)
end

model1opt (generic function with 4 methods)

In [3]:
max_val_list = [10 20 25 50]
min_dist = [.1 .25 .5 .75 1]

results1 = []

for mval in max_val_list
    for mdis in min_dist
        push!(results1, model1opt(mval, mdis, .006))
    end
end

rdf1 = DataFrame(results1)

CSV.write("method1.csv", rdf1)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 2.56402 - 0.00 seconds
Cgl0004I processed model has 1 rows, 484 columns (484 integer (484 of which binary)) and 484 elements
Cbc0012I Integer solution of -2.5538178 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search completed - best objective -2.553817829, took 0 iterations and 0 nodes (0.02 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -2.55382 to -2.55382
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and creat

"method1.csv"

### Model 2: Maximum summation of distance and coverage.

$max \sum{(b_i*v_i) + min\{\sqrt{(ex_j-x_i)^2+(ey_j-y_i)^2} \text{for all } j\}}$

In [21]:
function model2opt(max_units = 25, min_dist = 1, filter_density = 0)
    model2 = Model(Cbc.Optimizer)

    df= CSV.read("Data_With_Dist.csv", copycols=true)

    fdf = filter(row -> row[:value] > filter_density, df)
    
    m = size(fdf)[1]

    @variable(model2, b[1:m], Bin)

    @constraint(model2,sum(b[i] for i=1:m)<=max_units)

    for i=1:m
        @constraint(model2, fdf[9][i] >= min_dist*b[i])
    end

    @objective(model2, Max, sum((fdf[8][i]+fdf[9][i])*b[i] for i=1:m))
    model2;
    
    optimize!(model2)
    obj_val = objective_value(model2)
    
    soln = value.(b)
    sol = [fdf.Location[i] for (i,n) in enumerate(soln) if n == 1]
    
    return (max_units, min_dist, filter_density, obj_val, length(sol), sol)
end

model2opt (generic function with 4 methods)

In [22]:
max_val_list = [10 20 25 50]
min_dist = [.1 .25 .5 .75 1]

results2 = []

for mval in max_val_list
    for mdis in min_dist
        push!(results2, model2opt(mval, mdis, .006))
    end
end

rdf2 = DataFrame(results2)
CSV.write("method2.csv", rdf2)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 17.5058 - 0.00 seconds
Cgl0004I processed model has 1 rows, 484 columns (484 integer (484 of which binary)) and 484 elements
Cbc0012I Integer solution of -17.505841 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search completed - best objective -17.505841011, took 0 iterations and 0 nodes (0.02 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -17.5058 to -17.5058
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and crea

"method2.csv"

# Model 3

In [6]:
function model3opt(max_units = 25, min_dist = 1, filter_density = 0)
    model3 = Model(Cbc.Optimizer)

    df= CSV.read("Data_With_Dist.csv", copycols=true)

    fdf = filter(row -> row[:value] > filter_density, df)
#     electricity = CSV.read("electricity.csv", copycols=true)
    
    m = size(fdf)[1]
#     n = size(electricity)[1]

    @variable(model3, b[1:m], Bin)

    @constraint(model3,sum(b[i] for i=1:m)<=max_units)

    for i=1:m
        @constraint(model3, fdf[9][i] >= min_dist*b[i])
    end

    @objective(model3, Max, sum((fdf[8][i]+ .05*fdf[9][i])*b[i] for i=1:m))
    model3
    
    optimize!(model3)
    obj_val = objective_value(model3)
    
    soln = value.(b)
    sol = [fdf.Location[i] for (i,n) in enumerate(soln) if n == 1]
    
    return (max_units, min_dist, filter_density, obj_val, length(sol), sol)
end

model3opt (generic function with 4 methods)

In [7]:
# filter_list = [.005 .01 .025]
max_val_list = [10 20 25 50]
min_dist = [.1 .25 .5 .75 1]

results3 = []

for mval in max_val_list
    for mdis in min_dist
        push!(results3, model3opt(mval, mdis, .006))
    end
end

rdf3 = DataFrame(results3)

CSV.write("method3.csv", rdf3)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3.00136 - 0.00 seconds
Cgl0004I processed model has 1 rows, 484 columns (484 integer (484 of which binary)) and 484 elements
Cbc0012I Integer solution of -2.9949826 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search completed - best objective -2.994982617250001, took 0 iterations and 0 nodes (0.02 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -2.99498 to -2.99498
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and

"method3.csv"

In [8]:
function model4opt(max_units = 25, min_dist = 1, filter_density = 0)
    model4 = Model(Cbc.Optimizer)

    df= CSV.read("Data_With_Dist.csv", copycols=true)

    fdf = filter(row -> row[:value] > filter_density, df)
#     electricity = CSV.read("electricity.csv", copycols=true)
    
    m = size(fdf)[1]
#     n = size(electricity)[1]

    @variable(model4, b[1:m], Bin)

    @constraint(model4,sum(b[i] for i=1:m)<=max_units)

    for i=1:m
        @constraint(model4, fdf[9][i] >= min_dist*b[i])
    end

    @objective(model4, Max, sum((fdf[8][i]- .05*fdf[9][i])*b[i] for i=1:m))
    model4;
    
    optimize!(model4)
    obj_val = objective_value(model4)
    
    soln = value.(b)
    sol = [fdf.Location[i] for (i,n) in enumerate(soln) if n == 1]
    
    return (max_units, min_dist, filter_density, obj_val, length(sol), sol)
end

model4opt (generic function with 4 methods)

In [9]:
# filter_list = [.005 .01 .025]
max_val_list = [10 20 25 50]
min_dist = [.1 .25 .5 .75 1]

results4 = []

for mval in max_val_list
    for mdis in min_dist
        push!(results4, model4opt(mval, mdis, .006))
    end
end

rdf4 = DataFrame(results4)

CSV.write("method4.csv", rdf4)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 2.1324 - 0.00 seconds
Cgl0004I processed model has 1 rows, 59 columns (59 integer (59 of which binary)) and 59 elements
Cbc0012I Integer solution of -2.1189228 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -2.1189227832, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -2.11892 to -2.11892
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0

"method4.csv"